<a href="https://colab.research.google.com/github/VinayChaudhari1996/MISC/blob/main/NER_Model_Training_Validation_Evaluation_Testing_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Required Packges

In [2]:
!pip install sklearn_crfsuite gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.1 MB/

# Load the dataset

In [3]:
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import metrics



# Load the dataset
file_path = 'ner_dataset.csv'
df = pd.read_csv(file_path, encoding='latin1')

In [16]:
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
145097,Sentence: 6612,after,IN,O
145098,Sentence: 6612,15,CD,O
145099,Sentence: 6612,Cubans,NNS,B-gpe
145100,Sentence: 6612,were,VBD,O


In [4]:
# Fill missing sentence numbers
df['Sentence #'] = df['Sentence #'].fillna(method='ffill')

# Group the data by sentences
sentences = df.groupby('Sentence #')['Word'].apply(list).values
tags = df.groupby('Sentence #')['Tag'].apply(list).values

# Create a new dataframe for modeling
data = pd.DataFrame({'Sentence': sentences, 'Tag': tags})

# Display the first few rows of the new dataframe
data.head()

,Sentence,Tag
0,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
3,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]"
4,"[The, global, financial, crisis, has, left, Ic...","[O, O, O, O, O, O, B-org, O, O, O, O, O]"


# Splitting the dataset

In [17]:
# Split the data into train, validation, and test sets
train_sentences, temp_sentences, train_tags, temp_tags = train_test_split(data['Sentence'], data['Tag'], test_size=0.4, random_state=42)
val_sentences, test_sentences, val_tags, test_tags = train_test_split(temp_sentences, temp_tags, test_size=0.5, random_state=42)

# Convert the split data back to dataframes for consistency
train_data = pd.DataFrame({'Sentence': train_sentences, 'Tag': train_tags})
val_data = pd.DataFrame({'Sentence': val_sentences, 'Tag': val_tags})
test_data = pd.DataFrame({'Sentence': test_sentences, 'Tag': test_tags})

# Ensure all words in the sentences are strings
def ensure_string(sentences):
    return [[str(word) for word in sentence] for sentence in sentences]


In [19]:
train_data.shape , val_data.shape , test_data.shape

((3967, 2), (1322, 2), (1323, 2))

In [6]:

train_data['Sentence'] = ensure_string(train_data['Sentence'])
val_data['Sentence'] = ensure_string(val_data['Sentence'])
test_data['Sentence'] = ensure_string(test_data['Sentence'])

# Define feature extraction functions
def word2features(sent, i):
    word = sent[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return sent

# Prepare the data for CRF
X_train = [sent2features(s) for s in train_data['Sentence']]
y_train = [sent2labels(s) for s in train_data['Tag']]

X_val = [sent2features(s) for s in val_data['Sentence']]
y_val = [sent2labels(s) for s in val_data['Tag']]

X_test = [sent2features(s) for s in test_data['Sentence']]
y_test = [sent2labels(s) for s in test_data['Tag']]


In [7]:

# Initialize CRF model
# Averaged Perceptron
crf = sklearn_crfsuite.CRF(
    algorithm='ap',
    max_iterations=100,
    all_possible_transitions=False
)

# Train CRF model
crf.fit(X_train, y_train)

# Predict on validation set
y_pred = crf.predict(X_val)

# Evaluate model
f1_score = metrics.flat_f1_score(y_val, y_pred, average='weighted', labels=crf.classes_)
classification_report = metrics.flat_classification_report(y_val, y_pred, labels=crf.classes_)

print("F1 Score:", f1_score)
print("Classification Report:\n", classification_report)


F1 Score: 0.9596253484786343
Classification Report:
               precision    recall  f1-score   support

           O       0.99      0.99      0.99     24765
       B-geo       0.80      0.85      0.83       999
       B-org       0.71      0.65      0.68       537
       I-org       0.79      0.71      0.75       398
       B-tim       0.89      0.86      0.88       532
       I-tim       0.78      0.69      0.73       154
       B-art       0.00      0.00      0.00         9
       I-art       0.00      0.00      0.00         5
       B-gpe       0.86      0.86      0.86       451
       I-gpe       0.75      0.43      0.55        14
       B-per       0.76      0.75      0.76       511
       I-per       0.83      0.84      0.84       542
       I-geo       0.75      0.71      0.73       206
       B-eve       0.75      0.29      0.41        21
       B-nat       0.71      0.56      0.63         9
       I-eve       1.00      0.20      0.33        15
       I-nat       0.80     

# Test single record

In [15]:
# Define the function to predict NER tags for a given sentence
def predict_ner(sentence, model):
    # Preprocess the sentence
    words = sentence.split()

    # Extract features
    features = sent2features(words)

    # Predict tags
    predicted_tags = model.predict_single(features)

    return list(zip(words, predicted_tags))

# Example sentence
test_sentence = "Barak Obama born in Hawaii"

# Predict NER tags for the example sentence
predicted_ner_tags = predict_ner(test_sentence, crf)

# Print the results
print("Predicted NER tags:")
for word, tag in predicted_ner_tags:
    print(f"{word}: {tag}")


Predicted NER tags:
Barak: B-per
Obama: I-per
born: O
in: O
Hawaii: B-geo


# UI

In [14]:
import gradio as gr

# Function to predict NER tags for a given sentence using the trained CRF model
def predict_ner(sentence):
    # Preprocess the sentence
    words = sentence.split()

    # Extract features
    features = sent2features(words)

    # Predict tags
    predicted_tags = crf.predict_single(features)

    return {word: tag for word, tag in zip(words, predicted_tags)}

# Define the Gradio interface
iface = gr.Interface(
    fn=predict_ner,
    inputs=gr.Textbox(lines=2, placeholder="Enter a sentence here..."),
    outputs=gr.JSON(),
    title="Named Entity Recognition (NER)",
    description="Enter a sentence to get the NER tags for each word.",
    examples=[
        ["Barak Obama born in Hawaii"]
    ]
)

# Launch the Gradio interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://43083a7abd7245c364.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Optimized solution lbfgs

In [36]:

# Initialize CRF model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=450,
    c1=0.1,  # L1 regularization coefficient
    c2=0.1,  # L2 regularization coefficient
    all_possible_transitions=False
)

# Train CRF model
crf.fit(X_train, y_train)

# Predict on validation set
y_pred = crf.predict(X_val)

# Evaluate model
f1_score = metrics.flat_f1_score(y_val, y_pred, average='weighted', labels=crf.classes_)
classification_report = metrics.flat_classification_report(y_val, y_pred, labels=crf.classes_)

print("F1 Score:", f1_score)
print("Classification Report:\n", classification_report)


F1 Score: 0.9696718778914665
Classification Report:
               precision    recall  f1-score   support

           O       0.99      0.99      0.99    177010
       B-per       0.84      0.82      0.83      3364
       I-per       0.85      0.89      0.87      3433
       B-org       0.80      0.71      0.75      4039
       I-org       0.81      0.79      0.80      3375
       B-geo       0.86      0.91      0.88      7462
       B-tim       0.93      0.88      0.90      4072
       I-tim       0.82      0.78      0.80      1283
       B-gpe       0.96      0.94      0.95      3181
       I-geo       0.83      0.81      0.82      1519
       B-art       0.33      0.11      0.16        93
       B-eve       0.64      0.45      0.53        75
       I-eve       0.53      0.32      0.40        62
       I-art       0.07      0.01      0.02        74
       B-nat       0.63      0.42      0.51        45
       I-gpe       0.80      0.48      0.60        33
       I-nat       0.67     